In [ ]:
# ------------------------------------------------------------------------------
# Initial general settings
# ------------------------------------------------------------------------------
import os
import json
import pandas as pd
from datetime import datetime
import time
import timeit
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import drive
import math
from operator import add
from scipy.stats import rankdata
from tqdm.notebook import tqdm

from keras.applications.vgg16 import preprocess_input
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import RMSprop
!pip install -U efficientnet
import efficientnet.keras as efn
from tensorflow.keras import regularizers

# KERAS TUNER
# !pip install -q -U keras-tuner
# import kerastuner as kt
# from tensorboard.plugins.hparams import api as hp

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Set current working directory
cwd = '/content/drive/My Drive/Homeworks/ANNDL/HW1'
drive.mount('/content/drive')

# Set the seed for easy reproducibility
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

     |████████████████████████████████| 51kB 6.7MB/s 
Mounted at /content/drive


In [ ]:
# ------------------------------------------------------------------------------
# GPUs monitoring
# ------------------------------------------------------------------------------
# Set GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


After trying with hand-crafted CNN, we decide to move to transfer learning, since the preformance is significatively greater. We propose an ensemble method with 5 different strategies (mode, mean, maximum, weighted_mean, weighted_borda) building up 7 different models (m1 - m7). In the following chunck you can choose the models you want to use, if you want to train or load them by previous training sessions and the ensemble strategies you want to use.

In [ ]:
# ------------------------------------------------------------------------------
# Model and ensemble methods choosing
# ------------------------------------------------------------------------------

# Model to use
m1 = True
m2 = True # ex model 8
m3 = True
m4 = True
m5 = True
m6 = True
m7 = True

# Model to train
m1_train = True
m2_train = True # ex model 8
m3_train = True
m4_train = True
m5_train = True
m6_train = True
m7_train = True

models = "m" + "1"*m1 + "2"*m2 + "3"*m3 + "4"*m4 + "5"*m5 + "6"*m6 + "7"*m7

# Ensamble to use
mode = True
mean = True
maximum = True
weighted_mean = True
weighted_borda = True

In the following chunck we build the generators, we set the training/validation split (standard 20%) and we make a light image augmentation (seems not too determinant in this task) also introducing some gaussian noise helping to avoid overfitting. With more time, it would make sense to cross-validate in order to make the model wee all the validation data and find an average model.

To import the training images we used flow_from_directory after putting every training image in the properly sub-directory using the given JSON file with an easy python script.

In [ ]:
# ------------------------------------------------------------------------------
# Import training set & splitting [training, validation] = [0.8, 0.2]
# ------------------------------------------------------------------------------
# ImageDataGenerator
apply_data_augmentation = True   
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(
                                        rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        preprocessing_function=None,
                                        data_format='channels_last',
                                        validation_split=0.2,    
                                       )
else:
    train_data_gen = ImageDataGenerator(
    rescale=1./255,                             
    validation_split = 0.2)

valid_data_gen = ImageDataGenerator(
rescale=1./255,                             
validation_split = 0.2)

# Params
bs           = 32
num_classes  = 3
img_h        = 256
img_w        = 256

training_dir = os.path.join(cwd, 'training')

train_gen = train_data_gen.flow_from_directory(
    training_dir,                    
    batch_size=bs,
    class_mode='categorical',
    target_size=(img_h,img_w),
    shuffle=True, # it is nice to have them shuffled
    seed=SEED,
    subset='training')

valid_gen = valid_data_gen.flow_from_directory(
    training_dir,                    
    batch_size=bs,
    class_mode='categorical',
    target_size=(img_h,img_w),
    shuffle=True, # even in validation it is better to let it see diverse data at each epoch
    seed=SEED,
    subset='validation')

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_gen,               
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()

# Gaussian noise to help avoiding overfitting
train_dataset = tf.keras.layers.GaussianNoise(0.5)(train_dataset)

valid_dataset = tf.data.Dataset.from_generator(
    lambda: valid_gen,               
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()

Found 4492 images belonging to 3 classes.
Found 1122 images belonging to 3 classes.


We used **Keras tuner** to help us setting the hyperparameters (chunck not in this notebook) but, for simplicity, we also define the **Model_run** function to set on-the-fly some parameters as

*   **architecture**: architecture of base model for TL
*   **optimizer**: between Adam, RMSprop and Adadelta
*   **first_layer**: between an average pooling and a max pooling
*   **first_layer_size**: size of first layer
*   **dense_size**: size of dense layer
*   **dropout_size**: size of dropout
*   **L2_reg**: weight of L2 regularization in dense layer to help avoiding overfitting



In [ ]:
# ------------------------------------------------------------------------------
# Model run definition
# ------------------------------------------------------------------------------
def Model_run(architecture, optimizer, first_layer, first_layer_size, dense_size, dropout_size, L2_reg):

  # Params
  num_classes  = 3
  img_h        = 256
  img_w        = 256

  # Tansfer learning model
  if (architecture == "Xception"): # Model 1: Xception
    base_model = tf.keras.applications.Xception(
        input_shape = (img_h, img_w, 3),
        include_top = False, weights = 'imagenet') 

  if (architecture == "NASNetLarge"): # Model 2: NASNetLarge
    base_model = tf.keras.applications.NASNetLarge(
        input_shape = (img_h, img_w, 3),
        include_top =  False,
        weights = "imagenet")
    
  if (architecture == "InceptionV3"): # Model 3: Inception V3 (GoogleNet)
    base_model = InceptionV3(input_shape = (img_h, img_w, 3),
                             include_top = False,
                             weights = 'imagenet')
 
  if (architecture == "DenseNet201"): # Model 4: DenseNet201
    base_model = tf.keras.applications.DenseNet201(
        include_top = False,
        weights = "imagenet",
        input_shape = (img_h, img_w, 3))
  
  if (architecture == "ResNet152V2"): # Model 5: ResNet 152 V2
    base_model = tf.keras.applications.resnet_v2.ResNet152V2(input_shape=(img_h, img_w,3),
                                                             include_top=False,
                                                             weights="imagenet",
                                                             classes = num_classes)
 
  if (architecture == "InceptionResNetV2"): # Model 6: InceptionResNetV2
    base_model = tf.keras.applications.InceptionResNetV2(input_shape = (img_h, img_w, 3),
                                                         include_top = False,
                                                         weights = 'imagenet')

  if (architecture == "ResNet50"): # Model 7: ResNet50
    base_model = ResNet50(input_shape = (img_h, img_w,3),
                          include_top = False,
                          weights = "imagenet")
    
  if (architecture == "EfficientNet"): # Model 8: EfficientNet
    base_model = efn.EfficientNetB0(input_shape = (img_h, img_w, 3),
                                    include_top = False,
                                    weights = 'imagenet')
  
  # Hand-made NN
  if (first_layer == "AveragePooling2D"):
    x = tf.keras.layers.AveragePooling2D(pool_size=(first_layer_size,first_layer_size))(base_model.output)
  if (first_layer == "MaxPooling2D"):
    x = tf.keras.layers.MaxPooling2D(pool_size=(first_layer_size,first_layer_size))(base_model.output)
  
  x = tf.keras.layers.Flatten()(x)	

  if (L2_reg > 0):		                 
    x = tf.keras.layers.Dense(dense_size, activation='relu', kernel_regularizer=regularizers.l2(L2_reg))(x)
  else:
    x = tf.keras.layers.Dense(dense_size, activation='relu')(x)		  
  
  x = tf.keras.layers.Dropout(dropout_size)(x)			                             
  x = tf.keras.layers.Dense(units=num_classes, activation='softmax')(x)

  # Compiler params
  if (optimizer == "Adam"):
    optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4)
  if (optimizer == "RMSprop"):
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = 1e-4)
  if (optimizer == "Adadelta"): # we found to be a good choice after a lot of trial and error
    optimizer = tf.keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)

  loss = tf.keras.losses.CategoricalCrossentropy() # easy choice
  metrics = ["accuracy"]

  # Callbacks
  # Early stopping callback
  ES_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
  # LR adapter callback which help us to get in the right minimum
  LR_adapter_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='auto', min_delta=0.0001, cooldown=0)

  model = tf.keras.models.Model(base_model.input, x)

  model.compile(optimizer = optimizer,
                loss = loss,
                metrics = metrics)

  # model.summary()

  model.fit(train_dataset, 
              epochs           = 150, # normally stops much earlier
              steps_per_epoch  = len(train_gen), 
              validation_data  = valid_dataset,
              validation_steps = len(valid_gen),
              callbacks        = [ES_callback, LR_adapter_callback])
  
  return model

After a lot of trial and error we have choosen the following parameters in setting the model structure and training.

In [ ]:
# ------------------------------------------------------------------------------
# Train models
# ------------------------------------------------------------------------------
if (m1 and m1_train): # Model 1: Xception
  print("Model 1 training ...")
  model_01 = Model_run("Xception", "Adadelta", "MaxPooling2D", 7, 512, 0.5, 0.001)
  model_01.save(os.path.join(cwd, 'saved_model_01'))
  print("Model 1 saved")
if (m2 and m2_train): # Model 2: EfficientNet
  print("Model 2 training ...")
  model_02 = Model_run("EfficientNet", "Adadelta", "AveragePooling2D", 7, 218, 0.5, 0.001)
  model_02.save(os.path.join(cwd, 'saved_model_02'))
  print("Model 2 saved")
if (m3 and m3_train): # Model 3: Inception V3 (GoogleNet)
  print("Model 3 training ...")
  model_03 = Model_run("InceptionV3", "Adadelta", "AveragePooling2D", 5, 218, 0.5, 0.001)
  model_03.save(os.path.join(cwd, 'saved_model_03'))
  print("Model 3 saved")
if (m4 and m4_train): # Model 4: DenseNet201
  print("Model 4 training ...")
  model_04 = Model_run("DenseNet201", "Adadelta", "AveragePooling2D", 7, 218, 0.5, 0.001)
  model_04.save(os.path.join(cwd, 'saved_model_04'))
  print("Model 4 saved")
if (m5 and m5_train): # Model 5: ResNet 152 V2
  print("Model 5 training ...")
  model_05 = Model_run("ResNet152V2", "Adadelta", "AveragePooling2D", 7, 218, 0.5, 0)
  model_05.save(os.path.join(cwd, 'saved_model_05'))
  print("Model 5 saved")
if (m6 and m6_train): # Model 6: InceptionResNetV2
  print("Model 6 training ...")
  model_06 = Model_run("InceptionResNetV2", "Adadelta", "MaxPooling2D", 5, 512, 0.5, 0.001)
  model_06.save(os.path.join(cwd, 'saved_model_06'))
  print("Model 6 saved")
if (m7 and m7_train): # Model 7: ResNet50
  print("Model 7 training ...")
  model_07 = Model_run("ResNet50", "Adadelta", "AveragePooling2D", 7, 218, 0.5, 0.001)
  model_07.save(os.path.join(cwd, 'saved_model_07'))
  print("Model 7 saved")

In the following chuck you can load models from previous training session if you have setted

```
mx = True
mx_train = False
```



In [ ]:
# ------------------------------------------------------------------------------
# Load models
# ------------------------------------------------------------------------------
if (m1 and not m1_train): # Model 1: Xception
  print("Model 01 loading ...")
  model_01 = tf.keras.models.load_model(os.path.join(cwd, 'saved_model_01'))
  print("Model 01 loaded")
if (m2 and not m2_train): # Model 2: NASNetLarge
  print("Model 02 loading ...")
  model_02 = tf.keras.models.load_model(os.path.join(cwd, 'saved_model_02'))
  print("Model 02 loaded")
if (m3 and not m3_train): # Model 3: Inception V3 (GoogleNet)
  print("Model 03 loading ...")
  model_03 = tf.keras.models.load_model(os.path.join(cwd, 'saved_model_03'))
  print("Model 03 loaded")
if (m4 and not m4_train): # Model 4: DenseNet201
  print("Model 04 loading ...")
  model_04 = tf.keras.models.load_model(os.path.join(cwd, 'saved_model_04'))
  print("Model 04 loaded")
if (m5 and not m5_train): # Model 5: ResNet 152 V2
  print("Model 05 loading ...")
  model_05 = tf.keras.models.load_model(os.path.join(cwd, 'saved_model_05'))
  print("Model 05 loaded")
if (m6 and not m6_train): # Model 6: InceptionResNetV2
  print("Model 06 loading ...")
  model_06 = tf.keras.models.load_model(os.path.join(cwd, 'saved_model_06'))
  print("Model 06 loaded")
if (m7 and not m7_train): # Model 7: ResNet50
  print("Model 07 loading ...")
  model_07 = tf.keras.models.load_model(os.path.join(cwd, 'saved_model_07'))
  print("Model 07 loaded")
if (m8 and not m8_train): # Model 8: EfficientNet
  print("Model 08 loading ...")
  model_08 = tf.keras.models.load_model(os.path.join(cwd, 'saved_model_08'))
  print("Model 08 loaded")

Model 01 loading ...
Model 01 loaded
Model 03 loading ...
Model 03 loaded
Model 04 loading ...
Model 04 loaded
Model 05 loading ...
Model 05 loaded
Model 06 loading ...
Model 06 loaded
Model 07 loading ...
Model 07 loaded
Model 08 loading ...
Model 08 loaded


We make prediction, as well as with single models,  also on the following ensemble methods


*   **mode**: for each test image we take the majority of votes given by the 7 models
*   **mean**: for each test image we sum up the probabilities given by the 7 models and we take the class with highest probability
*   **maximum**: for each test image we take all the probabilities given by the 7 models and we take the maximum one
*   **weighted_mean**: as mean but every model is weighted based on its validation accuracy
*   **weighted_Borda**: as mean but every model previously gives its vote as a number between 1 and class_number based on its probabilities (Borda voting method), also weighted on its validation accuracy

Ensemble method is useful for unstable models that are different one from other and specialized on catch different features in images. In our case we always have obtained better results with ensemble method, despite, probably, our models are quite similar.

We found **weighted_mean** (95.1 % on test set) to be the best ensemble methods, taking into account exactly the single probabililties given by each model.

In [ ]:
# ------------------------------------------------------------------------------
# Prediction and saving results
# ------------------------------------------------------------------------------
# Function create_csv definition

def create_csv(results, models, ensemble, results_dir=cwd):
    if not os.path.exists(results_dir):
      os.mkdir(results_dir)
    csv_filename = 'results_' + datetime.now().strftime('%b%d_%H-%M-%S') + "_" + models + "_" + ensemble + '.csv'
    with open(os.path.join(results_dir, csv_filename), 'w') as f:
        f.write('Id,Category\n')
        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')
    return csv_filename

image_filenames = next(os.walk(os.path.join(cwd, 'test')))[2]

# Validation accuracies
if (m1):
  print("Model 1 validation accuracy:")
  val_acc_01 = model_01.evaluate(valid_dataset,steps=len(valid_gen),verbose=1)[1]
if (m2):
  print("Model 2 validation accuracy:")
  val_acc_02 = model_02.evaluate(valid_dataset,steps=len(valid_gen),verbose=1)[1]
if (m3):
  print("Model 3 validation accuracy:")
  val_acc_03 = model_03.evaluate(valid_dataset,steps=len(valid_gen),verbose=1)[1]
if (m4):
  print("Model 4 validation accuracy:")
  val_acc_04 = model_04.evaluate(valid_dataset,steps=len(valid_gen),verbose=1)[1]
if (m5):
  print("Model 5 validation accuracy:")
  val_acc_05 = model_05.evaluate(valid_dataset,steps=len(valid_gen),verbose=1)[1]
if (m6):
  print("Model 6 validation accuracy:")
  val_acc_06 = model_06.evaluate(valid_dataset,steps=len(valid_gen),verbose=1)[1]
if (m7):
  print("Model 7 validation accuracy:")
  val_acc_07 = model_07.evaluate(valid_dataset,steps=len(valid_gen),verbose=1)[1]

# Initialize results ensemble methods
if (maximum):
  results_maximum = {}
if (mean):
  results_mean = {}
if (mode):
  results_mode = {}
if (weighted_mean):
  results_weighted_mean = {}
if (weighted_borda):
  results_weighted_borda = {}

# Computed validation accuracy for weighted_mean
if (m1):
  results_m1 = {}
if (m2):
  results_m2 = {}
if (m3):
  results_m3 = {}
if (m4):
  results_m4 = {}
if (m5):
  results_m5 = {}
if (m6):
  results_m6 = {}
if (m7):
  results_m7 = {}

for image_name in tqdm(image_filenames):
  img = Image.open(os.path.join(cwd, 'test', image_name)).convert('RGB')
  img = img.resize((img_h,img_w))
  img_array = np.array(img)
  img_array = np.expand_dims(img_array, 0) 

  # Predictions probabilities for current img
  if (m1):
    pred_01 = model_01.predict(x=img_array/255.)
    results_m1[image_name] = np.argmax(pred_01)
  if (m2):
    pred_02 = model_02.predict(x=img_array/255.)
    results_m2[image_name] = np.argmax(pred_02)
  if (m3):
    pred_03 = model_03.predict(x=img_array/255.)
    results_m3[image_name] = np.argmax(pred_03)
  if (m4):
    pred_04 = model_04.predict(x=img_array/255.)
    results_m4[image_name] = np.argmax(pred_04)
  if (m5):
    pred_05 = model_05.predict(x=img_array/255.)
    results_m5[image_name] = np.argmax(pred_05)
  if (m6):
    pred_06 = model_06.predict(x=img_array/255.)
    results_m6[image_name] = np.argmax(pred_06)
  if (m7):
    pred_07 = model_07.predict(x=img_array/255.)
    results_m7[image_name] = np.argmax(pred_07)
  
  if (maximum):
    # ----------------------------------------------------------------------------
    # Maximum of choosen models
    # ----------------------------------------------------------------------------
    # Dictionary {model_number : max_probabily}
    max_val = {}
    
    if (m1):
      max_val[1] = np.max(pred_01)
    if (m2):
      max_val[2] = np.max(pred_02)
    if (m3):
      max_val[3] = np.max(pred_03)
    if (m4):
      max_val[4] = np.max(pred_04)
    if (m5):
      max_val[5] = np.max(pred_05)
    if (m6):
      max_val[6] = np.max(pred_06)
    if (m7):
      max_val[7] = np.max(pred_07)
    
    # Model with highest max_probability
    best = max(max_val, key=max_val.get)

    # Classification of best model
    if (best == 1):
      prediction_maximum = np.argmax(pred_01)
    elif (best == 2):
      prediction_maximum = np.argmax(pred_02)
    elif (best == 3):
      prediction_maximum = np.argmax(pred_03)
    elif (best == 4):
      prediction_maximum = np.argmax(pred_04)
    elif (best == 5):
      prediction_maximum = np.argmax(pred_05)
    elif (best == 6):
      prediction_maximum = np.argmax(pred_06)
    else:
      prediction_maximum = np.argmax(pred_07)

    results_maximum[image_name] = prediction_maximum

  if (mean):
    # ----------------------------------------------------------------------------
    # Mean of choosen models
    # ----------------------------------------------------------------------------
    # Sum of predictions of each model (NOT normalized)
    pred_sum = [0, 0, 0]
    
    if (m1):
      pred_sum = list(map(add, pred_sum, pred_01))
    if (m2):
      pred_sum = list(map(add, pred_sum, pred_02))
    if (m3):
      pred_sum = list(map(add, pred_sum, pred_03))
    if (m4):
      pred_sum = list(map(add, pred_sum, pred_04))
    if (m5):
      pred_sum = list(map(add, pred_sum, pred_05))
    if (m6):
      pred_sum = list(map(add, pred_sum, pred_06))
    if (m7):
      pred_sum = list(map(add, pred_sum, pred_07))

    # Class with highest sum of probability
    prediction_mean = np.argmax(pred_sum)
    results_mean[image_name] = prediction_mean

  if (mode):
    # ----------------------------------------------------------------------------
    # Mode of choosen models
    # ----------------------------------------------------------------------------
    # List of all probability lists
    pred_list = []
    
    if (m1):
      pred_list = [pred_list, pred_01]
    if (m2):
      pred_list = [pred_list, pred_02]
    if (m3):
      pred_list = [pred_list, pred_03]
    if (m4):
      pred_list = [pred_list, pred_04]
    if (m5):
      pred_list = [pred_list, pred_05]
    if (m6):
      pred_list = [pred_list, pred_06]
    if (m7):
      pred_list = [pred_list, pred_07]

    pred_list = pred_list[1:] # removes first empty list

    # Counter of votes for each class
    pred = [0, 0, 0]
    for elem in pred_list:
      pred[np.argmax(elem)] += 1

    results_mode[image_name]  = np.argmax(pred)

  if (weighted_mean):
    # ----------------------------------------------------------------------------
    # Weighted mean of choosen models
    # ----------------------------------------------------------------------------
    # Sum of predictions of each model (NOT normalized)
    pred_sum_weighted_mean = [0, 0, 0]
    
    if (m1):
      pred_sum_weighted_mean = list(map(add, pred_sum_weighted_mean, pred_01*val_acc_01))
    if (m2):
      pred_sum_weighted_mean = list(map(add, pred_sum_weighted_mean, pred_02*val_acc_02))
    if (m3):
      pred_sum_weighted_mean = list(map(add, pred_sum_weighted_mean, pred_03*val_acc_03))
    if (m4):
      pred_sum_weighted_mean = list(map(add, pred_sum_weighted_mean, pred_04*val_acc_04))
    if (m5):
      pred_sum_weighted_mean = list(map(add, pred_sum_weighted_mean, pred_05*val_acc_05))
    if (m6):
      pred_sum_weighted_mean = list(map(add, pred_sum_weighted_mean, pred_06*val_acc_06))
    if (m7):
      pred_sum_weighted_mean = list(map(add, pred_sum_weighted_mean, pred_07*val_acc_07))

    # Class with highest sum of probability
    results_weighted_mean[image_name] = np.argmax(pred_sum_weighted_mean)

  if (weighted_borda):
    # ----------------------------------------------------------------------------
    # Weighted Borda method applied to choosen models
    # ----------------------------------------------------------------------------
    # Sum of predictions of each model (NOT normalized)
    pred_sum_weighted_borda = [0, 0, 0]

    if (m1):
      pred_sum_weighted_borda = list(map(add, pred_sum_weighted_borda, np.inner(rankdata(pred_01), val_acc_01)))
    if (m2):
      pred_sum_weighted_borda = list(map(add, pred_sum_weighted_borda, np.inner(rankdata(pred_02), val_acc_02)))
    if (m3):
      pred_sum_weighted_borda = list(map(add, pred_sum_weighted_borda, np.inner(rankdata(pred_03), val_acc_03)))
    if (m4):
      pred_sum_weighted_borda = list(map(add, pred_sum_weighted_borda, np.inner(rankdata(pred_04), val_acc_04)))
    if (m5):
      pred_sum_weighted_borda = list(map(add, pred_sum_weighted_borda, np.inner(rankdata(pred_05), val_acc_05)))
    if (m6):
      pred_sum_weighted_borda = list(map(add, pred_sum_weighted_borda, np.inner(rankdata(pred_06), val_acc_06)))
    if (m7):
      pred_sum_weighted_borda = list(map(add, pred_sum_weighted_borda, np.inner(rankdata(pred_07), val_acc_07)))

    # Class with highest sum of probability
    results_weighted_borda[image_name] = np.argmax(list(pred_sum_weighted_borda))

Model 1 validation accuracy:
36/36 [==============================] - 191s 5s/step - loss: 0.3224 - accuracy: 0.9144
Model 3 validation accuracy:
36/36 [==============================] - 11s 317ms/step - loss: 0.3408 - accuracy: 0.9127
Model 4 validation accuracy:
36/36 [==============================] - 15s 408ms/step - loss: 0.2935 - accuracy: 0.9385
Model 5 validation accuracy:
36/36 [==============================] - 17s 485ms/step - loss: 0.2552 - accuracy: 0.9127
Model 6 validation accuracy:
36/36 [==============================] - 15s 427ms/step - loss: 0.3683 - accuracy: 0.9153
Model 7 validation accuracy:
36/36 [==============================] - 12s 338ms/step - loss: 0.3305 - accuracy: 0.9064
Model 8 validation accuracy:
36/36 [==============================] - 11s 302ms/step - loss: 0.4140 - accuracy: 0.8957


In this last chunck we create csv files, named with timestamp, models envolved and ensamble method choosen.

In [ ]:
# ------------------------------------------------------------------------------
# Create csv
# ------------------------------------------------------------------------------
# Create csv for single models
if (m1):
  csv_filename_m1 = create_csv(results_m1, "m1", "single", os.path.join(cwd, 'results'))
if (m2):
  csv_filename_m2 = create_csv(results_m2, "m2", "single", os.path.join(cwd, 'results'))
if (m3):
  csv_filename_m3 = create_csv(results_m3, "m3", "single", os.path.join(cwd, 'results'))
if (m4):
  csv_filename_m4 = create_csv(results_m4, "m4", "single", os.path.join(cwd, 'results'))
if (m5):
  csv_filename_m5 = create_csv(results_m5, "m5", "single", os.path.join(cwd, 'results'))
if (m6):
  csv_filename_m6 = create_csv(results_m6, "m6", "single", os.path.join(cwd, 'results'))
if (m7):
  csv_filename_m7 = create_csv(results_m7, "m7", "single", os.path.join(cwd, 'results'))
if (m8):
  csv_filename_m8 = create_csv(results_m8, "m8", "single", os.path.join(cwd, 'results'))

# Create csv for ensemble methods
if (maximum):
  csv_filename_maximum = create_csv(results_maximum, models, "maximum", os.path.join(cwd, 'results'))
if (mean):
  csv_filename_mean = create_csv(results_mean, models, "mean", os.path.join(cwd, 'results'))
if (mode):
  csv_filename_mode = create_csv(results_mode, models, "mode", os.path.join(cwd, 'results'))
if (weighted_mean):
  csv_filename_weighted_mean = create_csv(results_weighted_mean, models, "weighted_mean", os.path.join(cwd, 'results'))
if (weighted_borda):
  csv_filename_weighted_borda = create_csv(results_weighted_borda, models, "weighted_borda", os.path.join(cwd, 'results'))